In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import gmaps
from API_keys import google_api_key
import requests

In [2]:
# Loading Dataset

city_data = pd.read_csv('output_data/clean_city_data.csv')
city_data.drop(['Unnamed: 0'], axis=1, inplace=True)
city_data.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Berlevåg,70.86,29.09,50.0,66,64,23.04,NO,1591991156
1,Ushuaia,-54.80,-68.30,33.8,86,75,24.16,AR,1591991156
2,Rikitea,-23.12,-134.97,75.6,80,11,19.93,PF,1591991156
3,Mar del Plata,-38.00,-57.56,62.6,45,11,10.29,AR,1591990823
4,Mataura,-46.19,168.86,39.0,90,27,5.01,NZ,1591991157


In [3]:
# Creating Humidity Heatmap

gmaps.configure(api_key=google_api_key)
locations = city_data[['Latitude', 'Longitude']]
humidity = city_data['Humidity']

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity))
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Creating ideal weather df

ideal_weather_df = city_data[(city_data['Max Temp'] < 80) & \
                             (city_data['Max Temp'] >= 70) & \
                             (city_data['Wind Speed'] < 10) & (city_data['Cloudiness'] == 0)]

ideal_weather_df.reset_index(inplace=True)
ideal_weather_df = ideal_weather_df.drop(['index'], axis=1)
ideal_weather_df.head(20)

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sananduva,-27.95,-51.81,70.61,84,0,6.76,BR,1591991164
1,Port Keats,-14.25,129.55,71.74,58,0,7.67,AU,1591991167
2,Torbat-e Jām,35.24,60.62,74.26,19,0,5.73,IR,1591991172
3,Kiryat Gat,31.61,34.76,77.00,55,0,1.01,IL,1591990848
4,Angoche,-16.23,39.91,73.02,74,0,5.93,MZ,1591991176
5,Severnyy,50.68,36.55,73.40,60,0,6.71,RU,1591991178
6,Faya,18.39,42.45,73.40,73,0,2.68,SA,1591991182
7,Bukama,-9.20,25.85,72.30,43,0,2.89,CD,1591991185
8,São Filipe,14.90,-24.50,75.76,81,0,7.07,CV,1591990835
9,Yangi Marg`ilon,40.43,71.72,71.60,35,0,6.93,UZ,1591991193


In [5]:
# creating hotel df
hotel_df = pd.DataFrame(ideal_weather_df['City'])

In [6]:
# Creating Places layer

# Filling in values for hotel dataframe

for index, row in ideal_weather_df.iterrows():
    language = 'en'
    vtype = 'lodging'
    radius = '5000'
    vlat = str(ideal_weather_df.loc[index, 'Latitude'])
    vlng = str(ideal_weather_df.loc[index, 'Longitude'])
    url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={vlat},{vlng}&radius={radius}&type={vtype}&language={language}&key={google_api_key}'

    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {ideal_weather_df['City'][index]}.")
    response = requests.get(url).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Nearest Hotel'] = results[0]['name']
        hotel_df.loc[index, 'Nearest Hotel lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Nearest Hotel lon'] = results[0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Sananduva.
Closest hotel is Grupo Temtambém.
------------
Retrieving Results for Index 1: Port Keats.
Closest hotel is TDC Village (Hundred Man Camp).
------------
Retrieving Results for Index 2: Torbat-e Jām.
Closest hotel is ساختمان مسکونی فلورا.
------------
Retrieving Results for Index 3: Kiryat Gat.
Closest hotel is רגע בנחת.
------------
Retrieving Results for Index 4: Angoche.
Closest hotel is Guest House HESADA.
------------
Retrieving Results for Index 5: Severnyy.
Closest hotel is Full House North Mini-Hotel.
------------
Retrieving Results for Index 6: Faya.
Closest hotel is مقبرة ال مجمل.
------------
Retrieving Results for Index 7: Bukama.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: São Filipe.
Closest hotel is Tortuga B&B.
------------
Retrieving Results for Index 9: Yangi Marg`ilon.
Closest hotel is Ikathouse.
------------
Retrieving Results for Index 10: San Vicente.
Closest hotel is Complejo Turístico F

In [7]:
# Creating Places layer

# Cleaning hotel df

clean_hotel_df = hotel_df.dropna()
clean_hotel_df = clean_hotel_df.reset_index()
clean_hotel_df['Country'] = ideal_weather_df['Country']
clean_hotel_df.drop(['index'], axis=1)
clean_hotel_df

,index,City,Nearest Hotel,Nearest Hotel lat,Nearest Hotel lon,Country
0,0,Sananduva,Grupo Temtambém,-27.948863,-51.813781,BR
1,1,Port Keats,TDC Village (Hundred Man Camp),-14.237609,129.519258,AU
2,2,Torbat-e Jām,ساختمان مسکونی فلورا,35.241702,60.620323,IR
3,3,Kiryat Gat,רגע בנחת,31.648260,34.737376,IL
4,4,Angoche,Guest House HESADA,-16.225924,39.905828,MZ
5,5,Severnyy,Full House North Mini-Hotel,50.656890,36.560520,RU
6,6,Faya,مقبرة ال مجمل,18.389529,42.454887,SA
7,8,São Filipe,Tortuga B&B,14.880736,-24.492962,CD
8,9,Yangi Marg`ilon,Ikathouse,40.461418,71.728314,CV
9,10,San Vicente,Complejo Turístico Farelluk,-26.602405,-54.112158,UZ


In [8]:
# Creating Places layer

# Placing markers

# Creating a list for the values in desired columns
hotel_lst = clean_hotel_df['Nearest Hotel'].to_list()
city_lst = clean_hotel_df['City'].to_list()
country_lst = clean_hotel_df['Country'].to_list()

# Creating a list of strings that contain the message for the info box
master_lst = [f"""
 <dl>
 <dt>Name</dt><dd>{hotel_lst[i]}</dd>
 <dt>City</dt><dd>{city_lst[i]}</dd>
 <dt>Country</dt><dd>{country_lst[i]}</dd>
 </dl>
""" for i in range(len(clean_hotel_df))]

# Plotting the locations on top of the heatmap
locations2 = clean_hotel_df[['Nearest Hotel lat', 'Nearest Hotel lon']]
markers = gmaps.marker_layer(locations2, info_box_content = master_lst)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))